# SET UP
We start by setting up these two commands before starting witht he code.

In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wxigsls8
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wxigsls8
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=dc0b97ed48290e329b25af6e7f77b732cdd9917189c75485da11d1420e70b634
  Stored in directory: /tmp/pip-ephem-wheel-cache-4zrr7vs4/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# SEQUENTIAL CODE
Let us start with the sequential cod for this. I use the same code I used for assignment 2, however I make a small change with the 1D notation.


In [73]:
%%cu
# include <stdio.h>
#include <time.h>
#include <stdlib.h>

void matrixMulSequential(float *M, float *N, float *P, int i, int j, int k) {
    for (int x = 0; x < i; ++x) {
        for (int y = 0; y < k; ++y) {
            P[x * k + y] = 0;
            for (int z = 0; z < j; ++z) {
                P[x * k + y] += M[x * j + z] * N[z * k + y];
            }
        }
    }
}

int main() {
    // Matrix dimensions
    int i = 1024;
    int j = 1024;
    int k = 1024;

    float *M, *N, *P;
    M = (float *)malloc(i * j * sizeof(float));
    N = (float *)malloc(j * k * sizeof(float));
    P = (float *)malloc(i * k * sizeof(float));

    double times=0;
    int time =10;
    while(time>0){

    // Intilaizing the input matrices M and N with random values between 0 and 10
    for (int x = 0; x < i * j; ++x) {
        M[x] = ((float) rand() /RAND_MAX) *10.0;
    }
    for (int x = 0; x < j * k; ++x) {
        N[x] = ((float) rand() /RAND_MAX) *10.0;
    }

    clock_t start, end;
    double time_ellapsed;
    start = clock();

    matrixMulSequential(M, N, P, i, j, k);

    end = clock();
    time_ellapsed = ((double)(end - start) / CLOCKS_PER_SEC);
    printf("Time elapsed in the sequential code is: %f seconds\n", time_ellapsed);

    times+=time_ellapsed;
    time--;
    }
    printf("The average time spent in sequential code is: %f seconds",times/10 );

    // Free allocated memory
    free(M);
    free(N);
    free(P);

    return 0;
}

KeyboardInterrupt: ignored

# CODE 1: REGULAR PARALLELIZATION
Once set up, we cant start with the code. We start by the first code, which uses regular parallelization.

In [70]:
%%cu
# include <stdio.h>
#include <time.h>
#include <stdlib.h>

//matrix multiplication  kernel
__global__ void MatrixMulKernel(float* M, float* N, float* P, int i, int j, int k) {

 int Row = blockIdx.y*blockDim.y+threadIdx.y; //row and column of P
 int Col = blockIdx.x*blockDim.x+threadIdx.x;

 if ((Row < i) && (Col < k)) {
  // each thread computes one element of the block
  for (int z = 0; z < j; ++z) {
    P[Row*k+Col] += M[Row*j+z]*N[z*k+Col];
  }
 }

}

int main() {
    // Matrix dimensions
    int k = 1024;
    int j = 1024;
    int i = 1024;

    float *M, *N, *P;
    cudaMallocManaged( &M, i*j*sizeof(float));
    cudaMallocManaged( &N, j*k*sizeof(float));
    cudaMallocManaged( &P, i*k*sizeof(float));

    double times=0;
    int time =10;
    while(time>0){

    // Intilaizing the input matrices M and N with random values between 0 and 10
    for (int z = 0; z < i * j; ++z) {
        M[z]= ((float) rand() /RAND_MAX) *10.0;
        //M[z]=z;
    }
    for (int z = 0; z < j * k; ++z) {
        N[z]= ((float) rand() /RAND_MAX) *10.0;
        //N[z]=z;
    }

    clock_t start, end;
    double time_ellapsed;
    start = clock();

    // set grid, block and threads
    dim3 threadsPerBlock(32, 32); //16 x 16 block size, i.e 256 threads per block

    float blockPerGrid_x= (i + threadsPerBlock.x - 1) / threadsPerBlock.x;
    float blockPerGrid_y= (k + threadsPerBlock.y - 1) / threadsPerBlock.y;
    dim3 blocksPerGrid( blockPerGrid_x, blockPerGrid_y);

    // Launch the CUDA kernel
    MatrixMulKernel<<<blocksPerGrid, threadsPerBlock>>>(M, N, P, i, j, k);
    cudaDeviceSynchronize();

    end = clock();
    time_ellapsed = ((double) (end - start)/ CLOCKS_PER_SEC);

    printf("Time ellapsed in the parallelized code using block dimension [32x32] is: %f seconds\n",time_ellapsed);

    /*for (int z = 0; z < i; ++z) {
        for (int x = 0; x < k; ++x) {
            printf("%f ", P[z * k + x]);
        }
        printf("\n");
    }*/
    times+=time_ellapsed;
    time--;
    }

    printf("The average time spent in parallelized code is: %f seconds",times/10 );
    // Free allocated memory
    cudaFree(M);
    cudaFree(N);
    cudaFree(P);

    return 0;
}

Time ellapsed in the parallelized code using block dimension [32x32] is: 0.017386 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.017567 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016290 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016142 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016064 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016220 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016273 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016129 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016277 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.016214 seconds
The average time spent in parallelized code is: 0.016456 seconds


In [66]:
%%cu
# include <stdio.h>
#include <time.h>
#include <stdlib.h>
#define TILE_WIDTH 32

__global__ void MatrixMulTileKernel(float* M, float* N, float* P, int i, int j, int k) {
  __shared__ float ds_M[TILE_WIDTH][TILE_WIDTH];
  __shared__ float ds_N[TILE_WIDTH][TILE_WIDTH];
  int bx = blockIdx.x; int by = blockIdx.y;
  int tx = threadIdx.x; int ty = threadIdx.y;
  int Row = by * blockDim.y + ty;
  int Col = bx * blockDim.x + tx;

  // Loop over the M and N tiles required to compute the P element
  for (int p = 0; p < j/TILE_WIDTH; ++p) {
    ds_M[ty][tx] = M[Row * j + p * TILE_WIDTH + tx];
    ds_N[ty][tx] = N[(p * TILE_WIDTH + ty) * k + Col];
    __syncthreads();
    for (int z = 0; z < TILE_WIDTH; ++z){
        P[Row * k + Col] += ds_M[ty][z] * ds_N[z][tx];
    }
    __syncthreads();
  }
}

int main() {
  // Set dimensions
  int i = 1024;
  int j = 1024;
  int k = 1024;

  float *M, *N, *P;
  cudaMallocManaged(&M, i * j * sizeof(float));
  cudaMallocManaged(&N, j * k * sizeof(float));
  cudaMallocManaged(&P, i * k * sizeof(float));

  double times=0;
  int time =10;
  while(time>0){

  // Intilaizing the input matrices M and N with random values between 0 and 10
  for (int z = 0; z < i * j; ++z) {
      M[z]= ((float) rand() /RAND_MAX) *10.0;
      //M[z]=z;
  }
  for (int z = 0; z < j * k; ++z) {
      N[z]= ((float) rand() /RAND_MAX) *10.0;
      //N[z]=z;
  }
  clock_t start, end;
  double time_ellapsed;
  start = clock();

  // set grid, block and threads
  dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
  dim3 blocksPerGrid((k + TILE_WIDTH - 1) / TILE_WIDTH, (i + TILE_WIDTH - 1) / TILE_WIDTH);

  // Launch the CUDA kernel
  MatrixMulTileKernel<<<blocksPerGrid, threadsPerBlock>>>(M, N, P, i, j, k);
  cudaDeviceSynchronize();

  end = clock();
  time_ellapsed = ((double) (end - start)/ CLOCKS_PER_SEC);

  printf("Time ellapsed in the parallelized code using block dimension [32x32] is: %f seconds\n",time_ellapsed);
  times+=time_ellapsed;
  time--;
  }
   printf("The average time spent in parallelized code is: %f seconds",times/10 );

  // Free allocated memory
  cudaFree(M);
  cudaFree(N);
  cudaFree(P);

  return 0;
}


Time ellapsed in the parallelized code using block dimension [32x32] is: 0.002117 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001569 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001446 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001512 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001499 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001526 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001511 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001593 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001483 seconds
Time ellapsed in the parallelized code using block dimension [32x32] is: 0.001474 seconds
The average time spent in parallelized code is: 0.001573 seconds
